

https://www.tabnews.com.br/GabrielSozinho/documentacao-da-api-do-tabnews


GET {{BaseUrl}}/contents?page={pagina}&per_page={porPagina}&strategy={estrategia}

https://www.tabnews.com.br/api/v1

GET {{BaseUrl}}/{user}?page={pagina}&per_page={porPagina}&strategy={estrategia}

GET {{BaseUrl}}/contents/{user}/{slug}

## IMAGEM COM OS COMANDOS PASSADO NA URL QUE DEFINEM PAGINAÇÃO E ORDENAÇÃO

![Captura de tela 2025-03-12 012656.png](./Captura de tela 2025-03-12 012656.png "Captura de tela 2025-03-12 012656.png")




In [0]:
import requests
import pandas as pd
import datetime
import json
import time


## KWARGS PASSAR OS PARAMETROS DA URL DA API DENTRO DO RESPONSE ENCURTAR CAMINHO. EX> AO INVES DE 
## url = "https://www.tabnews.com.br/api/v1/contents/?page=1&per_page=100&strategy=new"
## url = "https://www.tabnews.com.br/api/v1/contents/" FICARIA ASSIM


def get_response(**kwargs):
    url = "https://www.tabnews.com.br/api/v1/contents/"  # Define a URL base da API
    resp = requests.get(url, params=kwargs)  # Faz a requisição GET passando os parâmetros dinamicamente
    return resp  # Retorna a resposta da requisição

def save_data(data, option='json'):
    now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")  # Gera um timestamp para nome do arquivo

    if option == 'json':
        with open(f"/dbfs/mnt/tabnews/{now}.json", "w") as open_file:  # Define o caminho no DBFS e abre o arquivo para escrita
            json.dump(data, open_file)  # Salva os dados no formato JSON

    elif option == 'dataframe':
        df = pd.DataFrame(data)  # Converte os dados em um DataFrame do Pandas
        df.to_parquet(f"/dbfs/mnt/tabnews/{now}.parquet", index=False)  # Salva os dados no formato Parquet

page = 1  # Inicializa a variável de controle da paginação
while True:
    print(page)  # Exibe o número da página atual no console para acompanhar a execução

    resp = get_response(page=page, per_page=100, strategy="new")  # Faz a requisição para a API com os parâmetros
    if resp.status_code == 200:  # Verifica se a requisição foi bem-sucedida
        data = resp.json()  # Converte a resposta para JSON
        save_data(data)  # Salva os dados no formato especificado

        if len(data) < 100:  # Se a quantidade de registros for menor que 100, significa que não há mais páginas disponíveis
            break  # Encerra o loop
        
        page += 1  # Incrementa o número da página para a próxima requisição
        time.sleep(5)  # Aguarda 2 segundos entre as requisições para evitar sobrecarga na API

    else:
        print(resp.status_code)  # Exibe o código de status da resposta em caso de erro
        print(resp.json())  # Exibe o conteúdo da resposta de erro para depuração
        time.sleep(60 * 15)  # Em caso de erro na requisição, espera 5 minutos antes de tentar novamente


In [0]:
## dbutils.fs.mkdirs("dbfs:/mnt/tabnews")


In [0]:

dbutils.fs.ls("dbfs:/mnt/tabnews")